# Visualization notebook

This notebook contains the k-NN model, as well as means for generating plots. The classifier is due to be moved into `classifier.py` in the future

-------

## Imports and functions

In [1]:
import utils
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

# 1,071,477 authors

# Carve a subset from MUD and store in database

In [9]:
import ijson
import sqlite3


data = ijson.parse(open("data/mud/full/raw_mud/raw_all/data.jsonl"),multiple_values=True)


i = 0
STOP = 20
d = {}
for prefix, event, value in data:
    pass
    
    
    
    
    
    # i += 1
    # if i == STOP:
    #     break



 start_map None 

 map_key syms 

syms start_array None 

syms.item string I dash attack waaaayyyyy too often 

syms.item string SO HAPPY ARMADA CAME THROUGH AGAINST THE PUNK ASS PUFF 

syms.item string Anyone have a replay of this? I missed it 

syms.item string You look like a dead goldfish that was left in its tank for so long it started to grow mold 

syms.item string Poké Floats was legal??? Better not tell twitch that 

syms.item string Were you homeschooled for how to do your hair then? 

syms.item string Call me insane, but I've always wanted to know what having a period feels like firsthand. (Sometimes even thought about childbirth, too) Seriously, the amount of pain women have to endure is just unfair. 

syms.item string Probably tell them that they really can do anything if they set their mind to it. Also to all the Santa comments, I'll tell my kids right away that Santa is made up, and let them spread that to the other kids at school.

Just realized: I'm one evil motherfuck